!pip install -r requirements.txt

In [1]:
import os
import time
import requests

from dotenv import load_dotenv
from selenium import webdriver
from bs4 import BeautifulSoup
from mistralai import Mistral

load_dotenv()

True

In [2]:
# Environment variables
EMAIL_MONDE = os.getenv("EMAIL_MONDE")
PASSWORD_MONDE = os.getenv("PASSWORD_MONDE")
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
MODEL = "mistral-large-latest"

# URLs
URL_ARTICLE = "https://www.lemonde.fr/idees/article/2025/09/12/comment-la-politique-de-l-offre-detourne-l-argent-public-au-profit-des-plus-riches_6640595_3232.html"
URL_COMMENTAIRES = URL_ARTICLE + "?contributions&page="

In [3]:
URL_COMMENTAIRES + "15"

'https://www.lemonde.fr/idees/article/2025/09/12/comment-la-politique-de-l-offre-detourne-l-argent-public-au-profit-des-plus-riches_6640595_3232.html?contributions&page=15'

In [4]:
# Create a new instance of the Chrome driver
driver = webdriver.Firefox()

# Open Google
#driver.get("https://www.lemonde.fr/")
driver.get("https://secure.lemonde.fr/sfuser/connexion")

input_element = driver.find_element("id", "email")
input_element.send_keys(EMAIL_MONDE)

password_input = driver.find_element("id", "password")
password_input.send_keys(PASSWORD_MONDE)

validate_button = driver.find_element("id", "submit-button")
validate_button.click()

all_comments = []

for page in range(1, 16):
    print(f"Loading page {page}...")
    driver.get(URL_COMMENTAIRES + str(page))
    #time.sleep(2)  # Wait for 2 seconds to allow the page to load

    html = driver.page_source

    soup = BeautifulSoup(html)
    #soup = BeautifulSoup(html, 'html.parser')

    # Find all elements with class="comment_content"
    comment_contents = soup.find_all(class_="comment__content")

    comment_list = [comment.text for comment in comment_contents]

    all_comments.extend(comment_list)

# Close the browser
driver.quit()

Loading page 1...
Loading page 2...
Loading page 3...
Loading page 4...
Loading page 5...
Loading page 6...
Loading page 7...
Loading page 8...
Loading page 9...
Loading page 10...
Loading page 11...
Loading page 12...
Loading page 13...
Loading page 14...
Loading page 15...


In [5]:
len(all_comments)

437

In [6]:
comment_txt = "\n".join(all_comments)

In [7]:
PROMPT_ANALYSE = f"""Tu es un expert en analyse de texte. 
Tu vas analyser les commentaires suivants issus d'un article du journal Le Monde. Chaque commentaire est séparé par un saut de ligne.
Voici les commentaires à analyser :{comment_txt}"""

In [8]:
client = Mistral(api_key=MISTRAL_API_KEY)

chat_response = client.chat.complete(
    model= MODEL,
    messages = [
        {
            "role": "user",
            "content": PROMPT_ANALYSE,
        },
    ]
)
print(chat_response.choices[0].message.content)

Voici une analyse structurée des commentaires issus de l'article du *Monde* sur le livre dénonçant le "grand détournement" des fonds publics vers les ultra-riches et les grandes entreprises. Les réactions reflètent une diversité d'opinions, souvent polarisées, mais aussi des débats économiques et idéologiques profonds. Voici une synthèse des principaux thèmes et arguments, ainsi que des dynamiques sous-jacentes.

---

### **1. Thèmes centraux et arguments clés**
#### **a) Critique de la politique de l'offre et des aides aux entreprises**
- **Détournement des fonds publics** : Beaucoup de commentateurs dénoncent un système où les aides aux entreprises (270 milliards d'euros selon le livre) profitent surtout aux actionnaires et aux ultra-riches, sans garantie de retombées économiques pour la collectivité. Exemples :
  - *"Les exonérations de charges sociales et les niches fiscales sont des cadeaux sans contrepartie"* (plusieurs commentaires).
  - *"Le CICE (Crédit d'Impôt pour la Compéti

In [9]:
PROMPT_BOT = f"""Tu es un expert détection d'ingérence par des bot russes.
Parmi tous les commentaires suivantes, séparés par des retours à la ligne, indique celui qui est le plus à même d'être issu d'un bot russe :{comment_txt}"""

In [10]:
chat_response = client.chat.complete(
    model= MODEL,
    messages = [
        {
            "role": "user",
            "content": PROMPT_BOT,
        },
    ]
)
print(chat_response.choices[0].message.content)

Parmi tous les commentaires fournis, le commentaire qui semble le plus susceptible d'être issu d'un bot russe est le suivant :

"Tout le monde sait que les différents gouvernements sont à la solde des riches et en particulier l'actuel président. IL est temps de passer à la caisse, les riches pour le bien de la société."

Voici pourquoi :

1. **Ton Conspiratif** : Ce commentaire utilise un ton très accusatoire et généraliste ("Tout le monde sait"), typique des discours de désinformation visant à semer la discorde et le doute envers les institutions.

2. **Simplification Extrême** : Il simplifie de manière excessive une situation complexe en accusant directement "les riches" et "l'actuel président" sans fournir de détails ou de preuves.

3. **Appel à l'Action** : L'appel à "passer à la caisse" peut être interprété comme une incitation à l'action, ce qui est une tactique courante pour mobiliser les émotions et créer des divisions.

4. **Manque de Nuance** : Il n'y a aucune nuance ou recon